In [ ]:
%matplotlib inline

from bs4 import BeautifulSoup
from iexfinance.stocks import Stock

import iexfinance
import matplotlib
import matplotlib.pyplot as plt
import re
import requests
import numpy as np
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.webdriver import WebDriver
import warnings


warnings.filterwarnings('ignore')

# Constituent data using browser automation

In [9]:
def get_etf_holdings(etf_symbol):
    '''
    etf_symbol: str
    
    return: pd.DataFrame

    '''
    dfdata = []
    for etfs in etf_symbol:

        url = 'https://www.barchart.com/stocks/quotes/{}/constituents?page=all'.format(
            etfs)

        # Loads the ETF constituents page and reads the holdings table
        browser = WebDriver() # webdriver.PhantomJS()
        browser.get(url)
        html = browser.page_source
        soup = BeautifulSoup(html, 'html')
        table = get_table(soup)

        # Reads the holdings table line by line and appends each asset to a
        # dictionary along with the holdings percentage
        asset_dict = {}
        for row in table.select('tr')[1:-1]:
            try:
                cells = row.select('td')
            
                symbol = cells[0].get_text().strip()

                name = cells[1].text.strip()
                celltext = cells[2].get_text().strip()
                percent = celltext.rstrip('%')
                shares = cells[3].text.strip().replace(',', '')
                if symbol != "" and percent != 0.0:
                    asset_dict[symbol] = {
                        'name': name,
                        'percent': percent,
                        'shares': shares,
                    }
                output = pd.DataFrame(asset_dict).T.reset_index().rename(columns={'index':'symbol'})
                dfdata.append(output)
            except BaseException as ex:
                print(ex)
        browser.quit()
    
    return pd.concat(dfdata)

In [6]:
etf = ['SPY','PXI']

In [10]:
check = get_etf_holdings(etf)

,symbol,name,percent,shares
0,AAPL,Apple Inc,7.01,N/A
0,AAPL,Apple Inc,7.01,N/A
1,MSFT,Microsoft Corp,5.94,N/A
0,AAPL,Apple Inc,7.01,N/A
1,MSFT,Microsoft Corp,5.94,N/A
...,...,...,...,...
36,HCC,Warrior Met Coal Inc,1.60,N/A
37,PUMP,Propetro Holding Corp,1.53,N/A
38,AROC,Archrock Inc,1.41,N/A
39,NE,Noble Corp Plc,1.36,N/A


In [8]:
def get_table(soup):
    for t in soup.select('table'):
        header = t.select('thead tr th')
        if len(header) > 2:
            if (header[0].get_text().strip() == 'Symbol'
                and header[2].get_text().strip().startswith('% Holding')):
                return t
    raise Exception('could not find symbol list table')
    
# Scrapes ETF holdings from barchart.com
def get_etf_holdings(etf_symbol):
    '''
    etf_symbol: str
    
    return: pd.DataFrame
    '''
    url = 'https://www.barchart.com/stocks/quotes/{}/constituents?page=all'.format(
        etf_symbol)

    # Loads the ETF constituents page and reads the holdings table
    browser = WebDriver() # webdriver.PhantomJS()
    browser.get(url)
    html = browser.page_source
    soup = BeautifulSoup(html, 'html')
    table = get_table(soup)

    # Reads the holdings table line by line and appends each asset to a
    # dictionary along with the holdings percentage
    asset_dict = {}
    for row in table.select('tr')[1:-1]:
        try:
            cells = row.select('td')
          
            symbol = cells[0].get_text().strip()

            name = cells[1].text.strip()
            celltext = cells[2].get_text().strip()
            percent = celltext.rstrip('%')
            shares = cells[3].text.strip().replace(',', '')
            if symbol != "" and percent != 0.0:
                asset_dict[symbol] = {
                    'name': name,
                    'percent': percent,
                    'shares': shares,
                }
        except BaseException as ex:
            print(ex)
    browser.quit()
    return pd.DataFrame(asset_dict)

In [ ]:
constituent = get_etf_holdings('SPY')

# Preview of data

In [ ]:
constituent.T

# IEX price data

In [ ]:
def iex_charts(symbols):
    partlen = 99
    result = {}
    for i in range(0, len(symbols), partlen):
        charts = Stock(symbols[i:i+partlen],token="pk_0282ac374bcd487d8bf51aeac3662f40").get_chart(range='1m')
        if type(charts) == list:
            charts = {symbols[i]: charts}
        for symbol, data in charts.items():
            df = pd.DataFrame(data).reset_index()


            print(df.columns)
            df.date = pd.to_datetime(df.date)
            df.set_index('date', inplace=True)
            df.index.names = ['epoch']
            df.index = df.index.tz_localize('America/New_York')
            result[symbol] = df
    return result

def get_closes(constituents):
    symbols = list(constituents.columns)
    charts = iex_charts(symbols)
    return pd.DataFrame({symbol: df.close for symbol, df in charts.items()})    

In [ ]:
def simulate(etf_symbol, cash, weights):
    # weights = get_etf_holdings(etf_symbol)
    symbols = list(weights.keys())
    charts = iex_charts(symbols)
    initial_prices = pd.Series({symbol: df.open[0] for symbol, df in charts.items()})
    last_prices = pd.Series({symbol: df.close[-1] for symbol, df in charts.items()})

    
    # construct portfolio
    portfolio = pd.DataFrame({
        'weight': weights,
        'initial_price': initial_prices,
        'last_price': last_prices,
    })
    portfolio.fillna(0)
    portfolio['shares'] = cash * portfolio.weight / 100 // portfolio.initial_price
    # calculate return for each symbol
    portfolio['return'] = (portfolio.last_price - portfolio.initial_price) * portfolio.shares
    
    return portfolio

# Putting together, replicate SPY itself

In [ ]:
closes = get_closes(constituent)

In [ ]:
replicated_perf = ((closes - closes.iloc[0]) * constituent.loc['shares']).sum(axis=1) / (closes.iloc[0] * constituent.loc['shares']).sum()

In [ ]:
etf = iex_charts(['SPY'])['SPY']
original_perf = (etf.close - etf.close[0]) / etf.close[0]

In [ ]:
performance = pd.DataFrame({'replicated': replicated_perf, 'original': original_perf})

In [ ]:
plt.figure(figsize=(12, 8))
performance.plot(figsize=(12, 8))

# What if I have only 10K?

In [ ]:
# This is to recalculate the weights at day 1 (instead of using the latest weights that are affected by the price changes)
mktvals = (closes.iloc[0] * constituent.loc['shares'])
weights = mktvals / mktvals.sum()

In [ ]:
# Say I have $10K, how many shares would I have bought for each?
cash = 1e+4
new_shares = cash * weights // closes.iloc[0]
positions = pd.DataFrame({
    'shares': new_shares,
    'price': closes.iloc[0],
    'dollar': closes.iloc[0] * new_shares,
}).loc[new_shares > 0]
positions

In [ ]:
positions.dollar.sum()

In [ ]:
new_perf = ((closes - closes.iloc[0]) * new_shares).sum(axis=1) / (closes.iloc[0] * new_shares).sum()

In [ ]:
new_result = pd.DataFrame({
    'replicated': new_perf,
    'original': original_perf,
})

In [ ]:
new_result

In [ ]:
new_result.plot(figsize=(12, 8))